In [1]:
# # depending on your shell, you might need to add `\` before `[` and `]`.
# !pip install -q flwr[simulation]
# !pip install flwr_datasets[vision]

We will be using the _simulation_ mode in Flower, which allows you to run a large number of clients without the overheads of manually managing devices. This is achieved via the [Virtual Client Engine](https://flower.ai/docs/framework/how-to-run-simulations.html) in Flower. With simulation, you can dynamically scale your experiments whether you run the code on your laptop, a machine with a single GPU, a server with multiple GPUs os even on a cluster with multiple servers. The `Virtual Client Engine` handles everything transparently and it allows you to specify how many resources (e.g. CPU cores, GPU VRAM) should be assigned to each virtual client.


Flower is agnostic to your choice of ML Framework. Flower works with `PyTorch`, `Tensorflow`, `NumPy`, `🤗 Transformers`, `MXNet`, `JAX`, `scikit-learn`, `fastai`, `Pandas`. Flower also supports all major platforms: `iOS`, `Android` and plain `C++`. You can find a _quickstart-_ example for each of the above in the [Flower Repository](https://github.com/adap/flower/tree/main/examples) inside the `examples/` directory.

In this experiment we are going to use PyTorch, it comes pre-installed in your Collab runtime so there is no need to installed it again. If you wouuld like to install another version, you can still do that in the same way other packages are installed via `!pip`

# Preparing the experiment
We make use of a ResNet12 for the MNIST image classification task.



In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet12(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet12, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Change input channels from 3 to 1
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 3, stride=1)
        self.layer2 = self._make_layer(128, 3, stride=2)
        self.layer3 = self._make_layer(256, 3, stride=2)
        self.layer4 = self._make_layer(512, 3, stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


We'll be training the model in a Federated setting. In order to do that, we need to define two functions:

* `train()` that will train the model given a dataloader.
* `test()` that will be used to evaluate the performance of the model on held-out data, e.g., a training set.

In [3]:
import flwr as fl
DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)
def train(net, trainloader, epochs):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optim = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
    net.train()
    for _ in range(epochs):
        for batch in trainloader:
            images, labels = batch["image"].to(DEVICE), batch["label"].to(DEVICE)
            optim.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optim.step()


def test(net, testloader):
    """Validate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, loss = 0, 0.0
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data["image"].to(DEVICE), data["label"].to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(testloader.dataset)
    return loss, accuracy

/home/vaishnavi/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-07 11:52:19,196	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cuda using PyTorch 2.3.1+cu121 and Flower 1.9.0


## One Client, One Data Partition

To start designing a Federated Learning pipeline we need to meet one of the key properties in FL: each client has its own data partition. To accomplish this with the MNIST dataset, we are going to generate N random partitions, where N is the total number of clients in our FL system.

We can use [Flower Datasets](https://flower.ai/docs/datasets/) to effortlessly obtain an off-the-shelf partitioned dataset or partition one that isn't pre-partitioned. Let's choose MNIST.

In [4]:
from datasets import Dataset
from flwr_datasets import FederatedDataset
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

# Let's set a simulation involving a total of 100 clients
NUM_CLIENTS = 100

# Download MNIST dataset and partition the "train" partition (so one can be assigned to each client)
mnist_fds = FederatedDataset(dataset="mnist", partitioners={"train": NUM_CLIENTS})
# Let's keep the test set as is, and use it to evaluate the global model on the server
centralized_testset = mnist_fds.load_split("test")

/home/vaishnavi/.local/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for mnist contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mnist
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Let's create a function that returns a set of transforms to apply to our images

In [5]:
from torchvision.transforms import ToTensor, Normalize, Compose


def apply_transforms(batch):
    """Get transformation for MNIST dataset"""

    # transformation to convert images to tensors and apply normalization
    transforms = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])
    batch["image"] = [transforms(img) for img in batch["image"]]
    return batch

## Defining a Flower Client

You can think of a client in FL as an entity that owns some data and trains a model using this data. The caveat is that the model is being trained _collaboratively_ in Federation by multiple clients (sometimes up to hundreds of thousands) and, in most instances of FL, is sent by a central server.

A Flower Client is a simple Python class with four distinct methods:

* `fit()`: With this method, the client does on-device training for a number of epochs using its own data. At the end, the resulting model is sent back to the server for aggregation.

* `evaluate()`: With this method, the server can evaluate the performance of the global model on the local validation set of a client. This can be used for instance when there is no centralised dataset on the server for validation/test. Also, this method can be use to asses the degree of personalisation of the model being federated.

* `set_parameters()`: This method takes the parameters sent by the server and uses them to initialise the parameters of the local model that is ML framework specific (e.g. TF, Pytorch, etc).

* `get_parameters()`: It extract the parameters from the local model and transforms them into a list of NumPy arrays. This ML framework-agnostic representation of the model will be sent to the server.



* `prune_model()`: It applies structured pruning to the weights and bias of Linear and Convolution layers.

FlowerClient(fl.client.Client) requires custom serialization and deserialization functions. We serialize the model parameters using `scipy.sparse.csr_matrix()` function and deserialize it by converting the sparse matrix to csr. Serializing the parameters to sparse matrix helps in reducing the number of bytes communicated from clients to server.

In [6]:
import torch
import torch.nn.utils.prune as prune

def prune_model(model, structured=True, amount=0.1, dim=0, **kwargs):
    # Collect the parameters to prune
    parameters_to_prune = []
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            if structured:
                parameters_to_prune.append((module, 'weight'))
            else:
                parameters_to_prune.append((module, 'weight'))
                if module.bias is not None:
                    parameters_to_prune.append((module, 'bias'))

    if structured:
        # Apply global structured pruning
        for module, param in parameters_to_prune:
            prune.ln_structured(module, name=param, amount=amount, n=1, dim=dim, **kwargs)
    
    # Remove pruning reparameterization
    for module, param in parameters_to_prune:
        prune.remove(module, param)

In [7]:
from io import BytesIO
from typing import cast

import numpy as np
import scipy

from flwr.common.typing import NDArray, NDArrays, Parameters


def ndarrays_to_sparse_parameters(ndarrays: NDArrays) -> Parameters:
    """Convert NumPy ndarrays to parameters object."""
    tensors = [ndarray_to_sparse_bytes(ndarray) for ndarray in ndarrays]
    return Parameters(tensors=tensors, tensor_type="numpy.ndarray")


def sparse_parameters_to_ndarrays(parameters: Parameters) -> NDArrays:
    """Convert parameters object to NumPy ndarrays."""
    return [sparse_bytes_to_ndarray(tensor) for tensor in parameters.tensors]


def ndarray_to_sparse_bytes(ndarray: NDArray) -> bytes:
    """Serialize NumPy ndarray to bytes."""
    bytes_io = BytesIO()

    if len(ndarray.shape) > 1:
        # Flatten higher-dimensional array to 2D
        original_shape = ndarray.shape
        ndarray = ndarray.reshape(-1, ndarray.shape[-1])

        # Convert ndarray to sparse matrix
        sparse_matrix = scipy.sparse.csr_matrix(ndarray)

        np.savez(
            bytes_io,
            data=sparse_matrix.data,
            indices=sparse_matrix.indices,
            indptr=sparse_matrix.indptr,
            shape=sparse_matrix.shape,
            original_shape=original_shape,  # Store original shape for reshaping
            allow_pickle=False,
        )
    else:
        np.save(bytes_io, ndarray, allow_pickle=False)
    return bytes_io.getvalue()


def sparse_bytes_to_ndarray(tensor: bytes) -> NDArray:
    """Deserialize NumPy ndarray from bytes."""
    bytes_io = BytesIO(tensor)
    loader = np.load(bytes_io, allow_pickle=False)

    if "indptr" in loader:
        # Convert sparse matrix back to ndarray
        sparse_matrix = scipy.sparse.csr_matrix(
            (loader["data"], loader["indices"], loader["indptr"]),
            shape=loader["shape"]
        )
        ndarray_deserialized = sparse_matrix.toarray()

        # Reshape back to original shape if needed
        if "original_shape" in loader:
            original_shape = loader["original_shape"]
            ndarray_deserialized = ndarray_deserialized.reshape(original_shape)
    else:
        ndarray_deserialized = loader
    return cast(NDArray, ndarray_deserialized)

In [8]:
from typing import Dict, List, Optional, Tuple
from collections import OrderedDict

def get_parameters(model: torch.nn.ModuleList) -> List[np.ndarray]:
    """Get model weights as a list of NumPy ndarrays."""
    return [val.cpu().numpy() for _, val in model.state_dict().items()]


def set_parameters(model: torch.nn.ModuleList, params: List[np.ndarray]):
    """Set model weights from a list of NumPy ndarrays."""
    params_dict = zip(model.state_dict().keys(), params)
    state_dict = OrderedDict({k: torch.from_numpy(np.copy(v)) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

In [9]:
from flwr.common import (
    Code,
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    GetParametersIns,
    GetParametersRes,
    Status,
)


class FlowerClient(fl.client.Client):
    def __init__(self, cid, net, trainloader, valloader, pruning_rate):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.pruning_rate = pruning_rate
        # Determine device
        self.new_model = ResNet12(num_classes=10)
    def get_parameters(self, ins: GetParametersIns) -> GetParametersRes:
        print(f"[Client {self.cid}] get_parameters")

        # Get parameters as a list of NumPy ndarray's
        ndarrays: List[np.ndarray] = get_parameters(self.net)

        # Serialize ndarray's into a Parameters object using our custom function
        parameters = ndarrays_to_sparse_parameters(ndarrays)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return GetParametersRes(
            status=status,
            parameters=parameters,
        )

    def fit(self, ins: FitIns) -> FitRes:
        print(f"[Client {self.cid}] fit, config: {ins.config}")

        # Deserialize parameters to NumPy ndarray's using our custom function
        parameters_original = ins.parameters
        ndarrays_original = sparse_parameters_to_ndarrays(parameters_original)

        # Update local model, train, get updated parameters
        set_parameters(self.net, ndarrays_original)
        train(self.net, self.trainloader, epochs=1)

        prune_model(self.net, structured=True, amount=self.pruning_rate, dim=0)

        self.new_model.load_state_dict(self.net.state_dict())
        ndarrays_updated = get_parameters(self.new_model)

        # Serialize ndarray's into a Parameters object using our custom function
        parameters_updated = ndarrays_to_sparse_parameters(ndarrays_updated)
        # Save the sparse matrix to an .npz file
        bytes_sent = sum(len(tensor) for tensor in parameters_updated.tensors)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return FitRes(
            status=status,
            parameters=parameters_updated,
            num_examples=len(self.trainloader),
            metrics={"bytes sent" : bytes_sent},
        )

    def evaluate(self, ins: EvaluateIns) -> EvaluateRes:
        print(f"[Client {self.cid}] evaluate, config: {ins.config}")

        # Deserialize parameters to NumPy ndarray's using our custom function
        parameters_original = ins.parameters
        ndarrays_original = sparse_parameters_to_ndarrays(parameters_original)

        set_parameters(self.net, ndarrays_original)
        loss, accuracy = test(self.net, self.valloader)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return EvaluateRes(
            status=status,
            loss=float(loss),
            num_examples=len(self.valloader),
            metrics={"accuracy": float(accuracy)},
        )

`evaluate_fn()` is the function for server side evaluation that uses the centralized testset

In [10]:
def get_evaluate_fn(centralized_testset: Dataset):
    """This is a function that returns a function. The returned
    function (i.e. `evaluate_fn`) will be executed by the strategy
    at the end of each round to evaluate the stat of the global
    model."""

    def evaluate_fn(server_round: int, parameters, config):
        """This function is executed by the strategy it will instantiate
        a model and replace its parameters with those from the global model.
        The, the model will be evaluate on the test set (recall this is the
        whole MNIST test set)."""

        model = ResNet12(num_classes=10)

        # Determine device
        model.to(DEVICE)  # send model to device

        # set parameters to the model
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.from_numpy(np.copy(v)) for k, v in params_dict})
        model.load_state_dict(state_dict, strict=True)

        # Apply transform to dataset
        testset = centralized_testset.with_transform(apply_transforms)

        testloader = DataLoader(testset, batch_size=50)
        # call test
        loss, accuracy = test(model, testloader)
        return loss, {"accuracy": accuracy}

    return evaluate_fn

# Defining Server-Side Strategy
`FedSparse()`: strategy uses the FedAvg aggregation method. To change the serialization and deserialization here, we only need to reimplement the evaluate and aggregate_fit functions of FedAvg. The other functions of the strategy will be inherited from the super class FedAvg

In [11]:
from logging import WARNING
from typing import Callable, Dict, List, Optional, Tuple, Union

from flwr.common import FitRes, MetricsAggregationFn, NDArrays, Parameters, Scalar
from flwr.common.logger import log
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy import FedAvg
from flwr.server.strategy.aggregate import aggregate

WARNING_MIN_AVAILABLE_CLIENTS_TOO_LOW = """
Setting `min_available_clients` lower than `min_fit_clients` or
`min_evaluate_clients` can cause the server to fail when there are too few clients
connected to the server. `min_available_clients` must be set to a value larger
than or equal to the values of `min_fit_clients` and `min_evaluate_clients`.
"""


class FedSparse(FedAvg):
    def __init__(
        self,
        *,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn: Optional[
            Callable[
                [int, NDArrays, Dict[str, Scalar]],
                Optional[Tuple[float, Dict[str, Scalar]]],
            ]
        ] = None,
        on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        accept_failures: bool = True,
        initial_parameters: Optional[Parameters] = None,
        fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
    ) -> None:
        """Custom FedAvg strategy with sparse matrices.

        Parameters
        ----------
        fraction_fit : float, optional
            Fraction of clients used during training. Defaults to 0.1.
        fraction_evaluate : float, optional
            Fraction of clients used during validation. Defaults to 0.1.
        min_fit_clients : int, optional
            Minimum number of clients used during training. Defaults to 2.
        min_evaluate_clients : int, optional
            Minimum number of clients used during validation. Defaults to 2.
        min_available_clients : int, optional
            Minimum number of total clients in the system. Defaults to 2.
        evaluate_fn : Optional[Callable[[int, NDArrays, Dict[str, Scalar]], Optional[Tuple[float, Dict[str, Scalar]]]]]
            Optional function used for validation. Defaults to None.
        on_fit_config_fn : Callable[[int], Dict[str, Scalar]], optional
            Function used to configure training. Defaults to None.
        on_evaluate_config_fn : Callable[[int], Dict[str, Scalar]], optional
            Function used to configure validation. Defaults to None.
        accept_failures : bool, optional
            Whether or not accept rounds containing failures. Defaults to True.
        initial_parameters : Parameters, optional
            Initial global model parameters.
        """

        if (
            min_fit_clients > min_available_clients
            or min_evaluate_clients > min_available_clients
        ):
            log(WARNING, WARNING_MIN_AVAILABLE_CLIENTS_TOO_LOW)

        super().__init__(
            fraction_fit=fraction_fit,
            fraction_evaluate=fraction_evaluate,
            min_fit_clients=min_fit_clients,
            min_evaluate_clients=min_evaluate_clients,
            min_available_clients=min_available_clients,
            evaluate_fn=evaluate_fn,
            on_fit_config_fn=on_fit_config_fn,
            on_evaluate_config_fn=on_evaluate_config_fn,
            accept_failures=accept_failures,
            initial_parameters=initial_parameters,
            fit_metrics_aggregation_fn=fit_metrics_aggregation_fn,
            evaluate_metrics_aggregation_fn=evaluate_metrics_aggregation_fn,
        )

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an evaluation function."""
        if self.evaluate_fn is None:
            # No evaluation function provided
            return None

        # We deserialize using our custom method
        parameters_ndarrays = sparse_parameters_to_ndarrays(parameters)

        eval_res = self.evaluate_fn(server_round, parameters_ndarrays, {})
        if eval_res is None:
            return None
        loss, metrics = eval_res
        return loss, metrics

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        if not results:
            return None, {}
        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # We deserialize each of the results with our custom method
        weights_results = [
            (sparse_parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]

        # We serialize the aggregated result using our custom method
        parameters_aggregated = ndarrays_to_sparse_parameters(
            aggregate(weights_results)
        )

        # Aggregate custom metrics if aggregation fn was provided
        metrics_aggregated = {}
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)
        elif server_round == 1:  # Only log this warning once
            log(WARNING, "No fit_metrics_aggregation_fn provided")

        return parameters_aggregated, metrics_aggregated

* `weighted_average()`: This function is used to aggregate the evaluation metrics returned by client `evaluate()` method. It calculates the average accuracy.

* `aggregate_fit_metrics()`: This function aggregates custom fit metrics from clients to calculate the average bytes sent.

In [12]:
from flwr.common import Metrics

def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """Aggregation function for (federated) evaluation metrics, i.e. those returned by
    the client's evaluate() method."""
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}

def aggregate_fit_metrics(metrics_list: List[Tuple[int, Dict[str, Scalar]]]) -> Dict[str, Scalar]:
    """Aggregate custom fit metrics from clients to calculate the average bytes sent.

    Args:
        metrics_list (List[Tuple[int, Dict[str, Scalar]]]): List of tuples, where each tuple
        contains the number of examples and a dictionary of metrics from a client.

    Returns:
        Dict[str, Scalar]: Aggregated metrics containing the average bytes sent.
    """
    total_bytes_sent = 0
    num_clients = len(metrics_list)

    for _, metrics in metrics_list:
        total_bytes_sent += metrics["bytes sent"]

    # Calculate the average bytes sent
    average_bytes_sent = total_bytes_sent / num_clients if num_clients > 0 else 0

    # Create the aggregated metrics dictionary
    aggregated_metrics = {
        "bytes sent": average_bytes_sent,
    }

    return aggregated_metrics


# Running the experiment
Here we initialize the FedSparse strategy defined above and runs the experiment from pruning rates 0.1 to 0.9. Calculated metrics are logged to a JSON file. Flower simulation is run using `fl.simulation.start_simulation()` method.

In [13]:
import json

strategy = FedSparse(evaluate_metrics_aggregation_fn = weighted_average,
                     fit_metrics_aggregation_fn = aggregate_fit_metrics,
                     evaluate_fn=get_evaluate_fn(centralized_testset))

client_resources = None
log_data = []

try:
    with open('Experimental data/MNIST_ResNet12_Structured.json', 'r') as f:
        log_data = json.load(f)
except FileNotFoundError:
    log_data = []

if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 3, "num_cpus":40}
for pruning_rate in np.arange(0.0, 0.01, 0.1):
    print(f"Running simulation with pruning rate: {pruning_rate}")
    def get_client_fn(dataset: FederatedDataset):
        """Return a function to construct a client.

        The VirtualClientEngine will execute this function whenever a client is sampled by
        the strategy to participate.
        """

        def client_fn(cid: str) -> FlowerClient:
            """Construct a FlowerClient with its own dataset partition."""

            # Let's get the partition corresponding to the i-th client
            client_dataset = dataset.load_partition(int(cid), "train")

            # Now let's split it into train (90%) and validation (10%)
            client_dataset_splits = client_dataset.train_test_split(test_size=0.1, seed=42)

            trainset = client_dataset_splits["train"]
            valset = client_dataset_splits["test"]

            # Now we apply the transform to each batch.
            trainloader = DataLoader(
                trainset.with_transform(apply_transforms), batch_size=32, shuffle=True
            )
            valloader = DataLoader(valset.with_transform(apply_transforms), batch_size=32)

            # Create the model and move it to the correct device
            net = ResNet12(num_classes=10).to(DEVICE)

            # Create and return client
            return FlowerClient(cid, net, trainloader, valloader, pruning_rate)

        return client_fn

    client_fn_callback = get_client_fn(mnist_fds)
    history = fl.simulation.start_simulation(
        strategy=strategy,
        client_fn=client_fn_callback,
        num_clients=2,
        config=fl.server.ServerConfig(num_rounds=50),
        client_resources=client_resources,
    )

    # Collect the metrics
    log_entry = {
        "pruning_rate": pruning_rate,
        "history_loss_distributed": history.losses_distributed,
        "history_loss_centralized": history.losses_centralized,
        "history_metrics_distributed_fit": history.metrics_distributed_fit,
        "history_metrics_distributed_evaluate": history.metrics_distributed,
        "history_metrics_centralized": history.metrics_centralized
    }
    log_data.append(log_entry)

# Write the collected log data to a JSON file
with open('Experimental data/MNIST_ResNet12_Structured.json', 'w') as f:
    json.dump(log_data, f, indent=4)

print("Metrics logged")


INFO :      Starting Flower simulation, config: num_rounds=50, no round_timeout


Running simulation with pruning rate: 0.0


2024-08-07 11:52:28,622	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'GPU': 3.0, 'object_store_memory': 200000000000.0, 'CPU': 40.0, 'node:10.8.0.3': 1.0, 'memory': 856336546816.0, 'node:__internal_head__': 1.0, 'accelerator_type:RTX': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_gpus': 3, 'num_cpus': 40}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 1 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client


(ClientAppActor pid=1199321) [Client 0] get_parameters


INFO :      Received initial parameters from one random client
INFO :      Evaluating initial global parameters
INFO :      initial parameters (loss, other metrics): 460.74843430519104, {'accuracy': 0.098}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)


...

(ClientAppActor pid=1199321) [Client 1] fit, config: {}
(ClientAppActor pid=1199321) [Client 0] fit, config: {}


INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      fit progress: (50, 22.876397872023517, {'accuracy': 0.9673}, 600.4486371381208)
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)


(ClientAppActor pid=1199321) [Client 0] evaluate, config: {}
(ClientAppActor pid=1199321) [Client 1] evaluate, config: {}


INFO :      aggregate_evaluate: received 2 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 50 round(s) in 602.37s
INFO :      	History (loss, distributed):
INFO :      		round 1: 9.84318208694458
INFO :      		round 2: 4.527614891529083
INFO :      		round 3: 1.4220208376646042
INFO :      		round 4: 1.9415481984615326
INFO :      		round 5: 0.9179914221167564
INFO :      		round 6: 0.5887374114245176
INFO :      		round 7: 0.3307669013738632
INFO :      		round 8: 0.49830277264118195
INFO :      		round 9: 0.33974439650774
INFO :      		round 10: 0.25369153171777725
INFO :      		round 11: 0.22908215131610632
INFO :      		round 12: 0.2432568520307541
INFO :      		round 13: 0.21312971785664558
INFO :      		round 14: 0.23580645583570004
INFO :      		round 15: 0.22688576392829418
INFO :      		round 16: 0.22975149005651474
INFO :      		round 17: 0.22707257512956858
INFO :      		round 18: 0.2186976559460163
INFO :      		round 19: 0.217946266755461

Metrics logged
